# Automation of  Data Collection from Whatsapp using Selenium

In [65]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import csv

# Function1  ---- Responsible for intiating the ChromeDriver

In [66]:
def initialize_driver():
    try:
        driver = webdriver.Chrome()
        driver.get("https://web.whatsapp.com")
    except Exception as e:
        print("Ensure That You Have a Internet Connection")
        return
    else:
        return driver

# Function2 --- Responsible for making the user to scan QRCode and returns after the successful Scanning of QRCode

In [67]:
def QRcodeScanner(driver):
    print("Please Scan the Whatsapp QRCode in new Automated Chrome Window")
    print("Please Press Enter after Successful Scanning of the code")
    input()
    flag = 1
    while(flag != 0):
        try:
            verification_element  = driver.find_element(By.ID,'initial_startup')
        except Exception as e:
            print("You Scanned the Code Succesfully")
            flag = 0
        else:
            print("Requested To Make Sure That You Scanned the Whatsapp QRCode")
            print("Please Press Enter after Successful Scanning of the code")
            input()
    return driver

# Function3 --- Returns the text message data of the specified user

In [59]:
def get_message_data(driver,contact):
    global SCROLL_TO, SCROLL_SIZE
    SCROLL_SIZE = 800
    SCROLL_TO = 0
    count = 0
    no_of_iterations = 20
    while(count < no_of_iterations):
    #     #Giving the contact specifically
    #     contact = 'Suresh Room'
        #Navigating through the specified Contact
        try:
            user = driver.find_element(By.XPATH,'//span[contains(@title, "{}")]'.format(contact))
            user.click()
        except Exception as e:
            print("The Contact Not found")
            print("Please Wait Scrolling the side pane>>>>>")
        else:
            print("The Contanct Found")
            break
        print('The Iteration : '+ str(count))
        count += 1
        print('>>> scrolling side pane')
        side_pane = driver.find_element(By.ID,'pane-side')
        driver.execute_script('arguments[0].scrollTop = '+str(SCROLL_TO), side_pane)
        sleep(3)
        SCROLL_TO += SCROLL_SIZE
        if(SCROLL_TO >=int(side_pane.get_attribute('scrollHeight'))):
            break
            
            
            
    scroll = 300
    prev_sum_scroll = 0
    now_sum_scroll = 0
    while(1):
        conversation_pane = driver.find_element(By.XPATH,"//div[@class='_5kRIK']")
        prev_sum_scroll = conversation_pane.get_attribute('scrollHeight')
        driver.execute_script('arguments[0].scrollTop = -' + str(scroll), conversation_pane)
        sleep(1)
        now_sum_scroll =  conversation_pane.get_attribute('scrollHeight')
        scroll = scroll + 500
        if(prev_sum_scroll == now_sum_scroll):
            break
        print(prev_sum_scroll)
        print(now_sum_scroll)
        conversation_pane.get_attribute('scrollHeight')


    #THIS CODE SASYS WHETHER THE MESSAGE IS SENT ONE OR RECEIVED ONE
    html_content = driver.page_source
    soup = BeautifulSoup(html_content,'html')
    a = soup.find_all('div',attrs = {'role':'row'})

    divs = []
    for x in a:
        temp =  x.find('div',attrs = {'class':'copyable-text'})
        if(temp != None):
            divs.append(temp)
    whole_data = []
    for div in divs:
        key = div.get('data-pre-plain-text')
        value = div.span.text
        whole_data.append([key,value])

    new_data = []
    for x in whole_data:
        # Extract timestamp, date, and name using regular expressions
        match = re.match(r'\[(.*?), (.*?)\] (.*)', x[0])

        if match:
            timestamp = match.group(1)
            date = match.group(2)
            name = match.group(3)
            result = [timestamp, date, name,x[1]]
            new_data.append(result)
        else:
            continue

    return new_data

# Function4 -- Responsible for making the json data to csv(structured format)

In [68]:
def prepare_csv_data(complete_data):
    keys = list(complete_data.keys())
    main_data = []
    for key in keys:
        for msg_data in complete_data[key]:
            main_data.append([key,msg_data[0],msg_data[1],msg_data[2],msg_data[3]])
    header = ['contanct_name','TimeStamp','Date','Sender','Message']
    with open("raw_msg_data.csv","w",encoding = "utf-8",newline='') as f:
        c_ob = csv.writer(f)
        c_ob.writerow(header)
        c_ob.writerows(main_data)
    print("The Data is succesfully written in raw_msg_data.csv")

# Main Code

In [69]:
driver = initialize_driver()


In [70]:
driver = QRcodeScanner(driver)

Please Scan the Whatsapp QRCode in new Automated Chrome Window
Please Press Enter after Successful Scanning of the code

You Scanned the Code Succesfully


In [71]:
#multiplecontancts
complete_data = {}
contact_list = ['metrics mahesh','Shyam sundar','Uday Room']
for contact in contact_list:
    data = get_message_data(driver,contact)
    complete_data[contact] = data

The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 0
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 1
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 2
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 3
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 4
>>> scrolling side pane
The Contanct Found
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 0
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 1
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 2
>>> scrolling side pane
The Contact Not found
Please Wait Scrolling the side pane>>>>>
The Iteration : 3
>>> scrolling side pane
The Contact Not found
Please Wait Sc

In [72]:
prepare_csv_data(complete_data)

The Data is succesfully written in raw_msg_data.csv


In [73]:
driver.quit()